In [3]:
import pandas as pd
melb_data = pd.read_csv('data/melb_data_ps.csv', sep=',')
melb_data.head()

,index,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,...,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount,Сoordinates
0,0,Abbotsford,85 Turner St,2,h,1480000.0,S,Biggin,3/12/2016,2.5,...,1,202.0,126.0,1970,Yarra,-37.7996,144.9984,Northern Metropolitan,4019,"-37.7996, 144.9984"
1,1,Abbotsford,25 Bloomburg St,2,h,1035000.0,S,Biggin,4/02/2016,2.5,...,0,156.0,79.0,1900,Yarra,-37.8079,144.9934,Northern Metropolitan,4019,"-37.8079, 144.9934"
2,2,Abbotsford,5 Charles St,3,h,1465000.0,SP,Biggin,4/03/2017,2.5,...,0,134.0,150.0,1900,Yarra,-37.8093,144.9944,Northern Metropolitan,4019,"-37.8093, 144.9944"
3,3,Abbotsford,40 Federation La,3,h,850000.0,PI,Biggin,4/03/2017,2.5,...,1,94.0,126.0,1970,Yarra,-37.7969,144.9969,Northern Metropolitan,4019,"-37.7969, 144.9969"
4,4,Abbotsford,55a Park St,4,h,1600000.0,VB,Nelson,4/06/2016,2.5,...,2,120.0,142.0,2014,Yarra,-37.8072,144.9941,Northern Metropolitan,4019,"-37.8072, 144.9941"


In [4]:
melb_df = melb_data.copy()
#melb_df.head()

In [6]:
melb_df = melb_df.drop(['index', 'Сoordinates'], axis=1)
#melb_df.head()

In [8]:
total_rooms = melb_df['Rooms'] + melb_df['Bedroom'] + melb_df['Bathroom']
melb_df['MeanRoomsSquare'] = melb_df['BuildingArea'] / total_rooms
#display(melb_df['MeanRoomsSquare'])

In [9]:
diff_area = melb_df['BuildingArea'] - melb_df['Landsize']
sum_area = melb_df['BuildingArea'] + melb_df['Landsize']
melb_df['AreaRatio'] = diff_area/sum_area
#display(melb_df['AreaRatio'])

In [10]:
#display(melb_df['Date'])
melb_df['Date'] = pd.to_datetime(melb_df['Date'])
#display(melb_df['Date'])
years_sold = melb_df['Date'].dt.year
#print(years_sold)
#print('Min year sold:', years_sold.min())
#print('Max year sold:', years_sold.max())
print('Mode year sold:', years_sold.mode()[0])

Mode year sold: 2017


In [12]:
#melb_df['MonthSale'] = melb_df['Date'].dt.month
#melb_df['MonthSale'].value_counts(normalize=True)

In [21]:
delta_days = melb_df['Date'] - pd.to_datetime('2016-01-01') 
#display(delta_days)
#display(delta_days.dt.days)

In [22]:
melb_df['AgeBuilding'] = melb_df['Date'].dt.year - melb_df['YearBuilt']
#display(melb_df['AgeBuilding'])

In [23]:
melb_df['WeekdaySale'] = melb_df['Date'].dt.dayofweek
weekend_count = melb_df[(melb_df['WeekdaySale'] == 5) | (melb_df['WeekdaySale'] == 6)].shape
#display(weekend_count)

In [38]:
ufo_data = pd.read_csv('data/ufo.csv')
ufo_data.head()

,City,Colors Reported,Shape Reported,State,Time
0,Ithaca,NaN,TRIANGLE,NY,6/1/1930 22:00
1,Willingboro,NaN,OTHER,NJ,6/30/1930 20:00
2,Holyoke,NaN,OVAL,CO,2/15/1931 14:00
3,Abilene,NaN,DISK,KS,6/1/1931 13:00
4,New York Worlds Fair,NaN,LIGHT,NY,4/18/1933 19:00


In [41]:
#ufo_data['Time'] = pd.to_datetime(ufo_data['Time'])
#ufo_data['Time'].dt.year.mode()

ufo_data['Date'] = ufo_data['Time'].dt.date
ufo_data[ufo_data['State'] == 'NV']['Date'].diff().dt.days.mean()


68.92932862190813

In [25]:
#melb_df['Address'].nunique()
def get_street_type(address):
    exclude_list = ['N', 'S', 'W', 'E']
    address_list = address.split(' ')
    street_type = address_list[-1]
    if street_type in exclude_list:
# Если переменная street_type является географической пометкой,
# переопределяем её на второй элемент с конца списка address_list.
        street_type = address_list[-2]
    return street_type

street_types = melb_df['Address'].apply(get_street_type)
#display(street_types)

KeyError: 'Address'

In [14]:
street_types.nunique()
popular_stypes = street_types.value_counts().nlargest(10).index
display(popular_stypes)

Index(['St', 'Rd', 'Ct', 'Dr', 'Av', 'Gr', 'Pde', 'Pl', 'Cr', 'Cl'], dtype='object')

In [15]:
melb_df['StreetType'] = street_types.apply(lambda x: x if x in popular_stypes else 'other')
#display(melb_df['StreetType'])

In [16]:
print(melb_df['StreetType'].nunique())
melb_df = melb_df.drop('Address', axis=1)

11


In [17]:
melb_df['WeekdaySale']

KeyError: 'WeekdaySale'

In [18]:
def get_weekend(weekday):
    if weekday == 5 or weekday == 6:
        return 1
    else:
        return 0
    
melb_df['Weekend'] = melb_df['WeekdaySale'].apply(get_weekend)
round(melb_df[melb_df['Weekend'] == 1]['Price'].mean(), 2)

KeyError: 'WeekdaySale'

In [19]:
#Задание 4.3
popular_seller = melb_df['SellerG'].value_counts().nlargest(49).index
#melb_df['StreetType'] = street_types.apply(lambda x: x if x in popular_stypes else 'other')
melb_df['SellerG'] = melb_df['SellerG'].apply(lambda x: x if x in popular_seller else 'other')

round(melb_df[melb_df['SellerG'] == 'Nelson']['Price'].min()/melb_df[melb_df['SellerG'] == 'other']['Price'].min(), 1)

1.3

5. Тип данных Category

In [20]:
# создаём пустой список
unique_list = []
# пробегаемся по именам столбцов в таблице
for col in melb_df.columns:
    # создаём кортеж (имя столбца, число уникальных значений)
    item = (col, melb_df[col].nunique(),melb_df[col].dtype) 
    # добавляем кортеж в список
    unique_list.append(item) 
# создаём вспомогательную таблицу и сортируем её
unique_counts = pd.DataFrame(
    unique_list,
    columns=['Column_Name', 'Num_Unique', 'Type']
).sort_values(by='Num_Unique',  ignore_index=True)
# выводим её на экран
display(unique_counts)

,Column_Name,Num_Unique,Type
0,Type,3,object
1,Method,5,object
2,Regionname,8,object
3,Rooms,9,int64
4,Bathroom,9,int64
5,Car,11,int64
6,StreetType,11,object
7,Bedroom,12,int64
8,MonthSale,12,int64
9,CouncilArea,33,object


In [27]:
#display(melb_df.info())
cols_to_exclude = ['Date', 'Rooms', 'Bedroom', 'Bathroom', 'Car'] # список столбцов, которые мы не берём во внимание
max_unique_count = 140 # задаём максимальное число уникальных категорий
for col in melb_df.columns: # цикл по именам столбцов
    if melb_df[col].nunique() < max_unique_count and col not in cols_to_exclude: # проверяем условие
        melb_df[col] = melb_df[col].astype('category') # преобразуем тип столбца
display(melb_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13580 entries, 0 to 13579
Data columns (total 26 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Suburb           13580 non-null  object        
 1   Rooms            13580 non-null  int64         
 2   Type             13580 non-null  category      
 3   Price            13580 non-null  float64       
 4   Method           13580 non-null  category      
 5   SellerG          13580 non-null  category      
 6   Date             13580 non-null  datetime64[ns]
 7   Distance         13580 non-null  float64       
 8   Postcode         13580 non-null  int64         
 9   Bedroom          13580 non-null  int64         
 10  Bathroom         13580 non-null  int64         
 11  Car              13580 non-null  int64         
 12  Landsize         13580 non-null  float64       
 13  BuildingArea     13580 non-null  float64       
 14  YearBuilt        13580 non-null  int64

None

In [33]:
#melb_df['Regionname'].cat.categories
#melb_df['Regionname'].cat.codes

#melb_df['Type'] = melb_df['Type'].cat.rename_categories({
#    'u': 'unit',
#    't': 'townhouse',
#    'h': 'house'
#})
#display(melb_df['Type'])

In [32]:
melb_df['Type'] = melb_df['Type'].cat.add_categories('flat')
new_houses_types = pd.Series(['unit', 'house', 'flat', 'flat', 'house'])
new_houses_types = new_houses_types.astype(melb_df['Type'].dtype)
display(new_houses_types)

0     unit
1    house
2     flat
3     flat
4    house
dtype: category
Categories (4, object): ['house', 'townhouse', 'unit', 'flat']

In [123]:
#melb_df.info()
melb_new = melb_df.copy()
#melb_new['Suburb']
#melb_new['Suburb'].value_counts()

#melb_freq = melb_new['Suburb'].value_counts().iloc[0:119]
#for st in melb_new['Suburb'].value_counts(): # цикл по именам столбцов
#    if st not in melb_freq: # проверяем условие
#       melb_new[st] = melb_new[st].astype('category') # преобразуем тип столбца
#melb_new['Suburb'][1]

#melb_new['Suburb'].value_counts().iloc[119:315].astype('category')
#melb_new['Suburb'].cat.categories


popular_sub = melb_new['Suburb'].value_counts().nlargest(119).index
melb_new['Suburb'] = melb_new['Suburb'].apply(lambda x: x if x in popular_sub else 'other')
melb_new['Suburb'].mode().astype('category')
melb_df.info()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13580 entries, 0 to 13579
Data columns (total 26 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Suburb           13580 non-null  object        
 1   Rooms            13580 non-null  int64         
 2   Type             13580 non-null  category      
 3   Price            13580 non-null  float64       
 4   Method           13580 non-null  category      
 5   SellerG          13580 non-null  category      
 6   Date             13580 non-null  datetime64[ns]
 7   Distance         13580 non-null  float64       
 8   Postcode         13580 non-null  int64         
 9   Bedroom          13580 non-null  int64         
 10  Bathroom         13580 non-null  int64         
 11  Car              13580 non-null  int64         
 12  Landsize         13580 non-null  float64       
 13  BuildingArea     13580 non-null  float64       
 14  YearBuilt        13580 non-null  int64